In [20]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy import stats

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


import warnings
warnings.filterwarnings("ignore")

In [21]:
def get_telco_data():
    '''

    checks if 'telco.csv' exists
    if it does it will load file with Pandas

    otherwise, it will connect to SQL server
    using get_db_url('telco_churn') from env.py
    and read_sql() from Pandas

    returns a dataframe

    '''
    path = 'telco.csv'
    file_exists = os.path.exists(path)
    if file_exists:

        df = pd.read_csv(path)

        return df

    else:
        url = env.get_db_url('telco_churn')
        sql_query = '''
                        SELECT
                            *
                        FROM
                            customers AS c
                            LEFT JOIN internet_service_types AS ist USING (internet_service_type_id)
                            LEFT JOIN customer_subscriptions AS csb USING (customer_id)
                            LEFT JOIN payment_types AS pt USING (payment_type_id)
                            LEFT JOIN contract_types AS ct USING (contract_type_id)
                            LEFT JOIN customer_churn AS ccr USING (customer_id)
                            LEFT JOIN customer_contracts AS ccn USING (customer_id)
                            LEFT JOIN customer_details AS cd USING (customer_id)
                            LEFT JOIN customer_payments AS cp USING (customer_id)
                            LEFT JOIN customer_signups AS cs USING (customer_id);
                    '''
        df = pd.read_sql(sql_query, url)
        df.to_csv('telco_churn.csv')
        return df

In [13]:
def clean_telco_data(df):
    import pandas as pd
    
    df = df.loc[:,~df.columns.duplicated()].copy()
    df['total_charges'] = (df.total_charges + '0').astype('float')
    df = df.drop(columns=['internet_service_type_id', 'contract_type_id', 'payment_type_id'])
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type'
                            ]],
                              drop_first=True)
    df = pd.concat( [df, dummy_df], axis=1 )
    
    return df

In [4]:
def split_telco_data(df, target='churn'):
    from sklearn.model_selection import train_test_split
    '''
    split_data will take in a single pandas dataframe
    it will split it into a train, validate, and test set
    and it will return three values:
    train, val, test (in this order) -- all pandas Dataframes
    '''
    train, test = train_test_split(df, 
                               train_size = 0.8,
                               random_state=1349,
                              stratify=df[target])
    train, val = train_test_split(train,
                             train_size = 0.8,
                             random_state=1349,
                             stratify=train[target])
    return train, val, test

In [6]:
def model_telco_data(df,target='churn_encoded'):
    columns_to_drop = ['customer_id','gender','partner','dependents','phone_service','paperless_billing','churn',\
              'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support',\
               'streaming_tv', 'streaming_movies', 'contract_type', 'internet_service_type', 'payment_type',\
                  'contract_type','churn_month','signup_date']
    columns_to_drop.append(target)
    
    X_df = df.drop(columns=columns_to_drop)
    y_df = df[target]

    return X_df, y_df

In [15]:
# acquiring, cleaning, and adding features to data
df = get_telco_data()
df = clean_telco_data(df)

# splitting data into train, validate, and test
train, validate, test = split_telco_data(df)

In [16]:
# saving unique customer id for later in the correct order to match predictions in CSV
train_cust, validate_cust, test_cust = train['customer_id'],validate['customer_id'],test['customer_id']

In [25]:
# Isolate Target Variable
X_train, y_train = model_telco_data(train)
X_validate, y_validate = model_telco_data(validate)

In [23]:
baseline = df['churn_encoded'].value_counts().idxmax()

In [26]:
logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')
logit.fit(X_train, y_train)

y_pred = logit.predict(X_train)
train_acc = pd.DataFrame()
train_acc['train_prediction']=y_pred
train_acc['actual']=y_train.tolist()
train_acc['baseline_prediction']=baseline
model_accuracy = (train_acc.train_prediction == train_acc.actual).mean()
baseline_accuracy = (train_acc.baseline_prediction == train_acc.actual).mean()

print(f'Logistic Regression accuracy: {model_accuracy:.2%} on training set')
print(f'           Baseline accuracy: {baseline_accuracy:.2%} on training set\n')


y_pred = logit.predict(X_validate)
validate_acc = pd.DataFrame()
validate_acc['validate_prediction']=y_pred
validate_acc['actual']=y_validate.tolist()
validate_acc['baseline_prediction']=baseline
model_accuracy = (validate_acc.validate_prediction == validate_acc.actual).mean()
baseline_accuracy = (validate_acc.baseline_prediction == validate_acc.actual).mean()

print(f'Logistic Regression accuracy: {model_accuracy:.2%} on validate set')
print(f'           Baseline accuracy: {baseline_accuracy:.2%} on validate set\n')

Logistic Regression accuracy: 33.41% on training set
           Baseline accuracy: 73.46% on training set

Logistic Regression accuracy: 33.36% on validate set
           Baseline accuracy: 73.47% on validate set

